In [74]:
import os
import pandas as pd
import numpy as np
import time

from py_package import db_conn as db
from datetime import datetime
from datetime import timedelta

from random   import choice

reload(db)

<module 'py_package.path_maker' from '/Users/szymonbocian/Documents/CallCenterStaffing/DataScripts/py_package/path_maker.py'>

In [75]:
#''' Check if file exists on path. Move existing file to ARCH. Create new input for calendar_fact.'''

connection_dict = { "user"     : "szymonbocian", \
                    "password" : "", \
                    "host"     : "localhost", \
                    "port"     : "5432", \
                    "database" : "dwh_call_center"
                  }

db = db.DbConn(connection_dict)

In [78]:
# connection to database and return list of column column_list
# print(db.run_sql("select * from stg.client_dim;"))

sql_dict = {
    "column_list"                  : "SELECT column_name FROM INFORMATION_SCHEMA.columns WHERE table_schema = 'stg' AND table_name = 'consultant_work_calendar_fact';",    
    "consultant_business_key_list" : "SELECT consultant_business_key FROM pro.consultant_dim WHERE active = True;",
    "office_business_key_list"     : "SELECT office_business_key FROM pro.office_dim WHERE active = True;",
    "week_list"                    : "SELECT MAX(start_of_week_mon_date), MAX(end_of_week_mon_date) FROM tech.date_dim WHERE iso_week_of_year = " + str(week_of_year) + " AND year_value = " + str(value_year) + ";"
}

for value in sql_dict.values():
    value = db.run_sql(value)

In [109]:
# df_cons = pd.DataFrame({
#     "consultant_business_key" : consultant_business_key_list,
#     "office_business_key" : np.random.choice(office_business_key_list, len(consultant_business_key_list)),
# })

start_time = time.mktime(datetime.timestamp(week_list[0][0]) + timedelta(hours = work_start_time)
print(start_time)
df_date = pd.DataFrame({
    "work_date" : pd.date_range(start = week_list[0][0], end = week_list[0][1])
})

# work_time = work_end_time - work_start_time



# df_time = pd.DataFrame({
#     "work_time" : pd.date_range('2019-01-10', periods = work_time, freq = '8')
# })
# df_time
# print(week_list[0][0])
# print(week_list[0][1])

TypeError: descriptor 'timestamp' requires a 'datetime.datetime' object but received a 'datetime.date'

In [51]:
# liczba_dni * liczba_godzin * liczba_pracowników * średnia_liczba_połączeń_na_godzinę
# 7 dni w tygodniu , od 6:00 do 22:00 ( 16 godzin )
# 100 konsultantów
# 5 - 8 połączeń na godzinę / konsultant
# dane z 4 tygodni i prognozowanie na ich podstawie
#tydzień
# 7*16*100*5     # = 56 000
# 7*16*100*8     # = 89 600
# 4 tygodnie     
# 4*7*16*100*5    # = 224 000
# 4*7*16*100*8    # = 358 400


358400